Implementation of Face Recognition assignment.
As the YOLO model is very computationally expensive to train, the pre-trained weights are already loaded for you to use.
Aim:-
1. Differentiate between face recognition and face verification
2. Implement one-shot learning to solve a face recognition problem
3. Apply the triplet loss function to learn a network's parameters in the context of face recognition
4. Explain how to pose face recognition as a binary classification problem
5. Map face images into 128-dimensional encodings using a pretrained model
6. Perform face verification and face recognition with these encodings

Note: Loading the "model.h5" on the local interpreter with python v3.9 and tensorflow version 2.9.1 throws a value error "ValueError:bad marshal data". It is recommended to run this assignment using google colab or use python v3.7.13 and tensorflow 2.8.2

In [34]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import pandas as pd
import tensorflow as tf
import PIL
from tensorflow.keras.models import model_from_json

%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Encoding Face Images into a 128-Dimensional Vector

Importing the model

In [36]:
# main folder with all the files. Please adapt this line to your use. 
ROOT_DIR = "/content/drive/MyDrive/Colab Notebooks/Coursera_NB/Week_04"
# path of the h5 files
fname = os.path.join(ROOT_DIR, 'keras-facenet-h5', 'facenet_keras.h5')
model = tf.keras.models.load_model(fname)

In [37]:
print(model.input)
print(model.output)

KerasTensor(type_spec=TensorSpec(shape=(None, 160, 160, 3), dtype=tf.float32, name='input_1'), name='input_1', description="created by layer 'input_1'")
KerasTensor(type_spec=TensorSpec(shape=(None, 128), dtype=tf.float32, name=None), name='Bottleneck_BatchNorm/batchnorm/add_1:0', description="created by layer 'Bottleneck_BatchNorm'")


In [38]:
# model summary
model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 160, 160, 3  0           []                               
                                )]                                                                
                                                                                                  
 Conv2d_1a_3x3 (Conv2D)         (None, 79, 79, 32)   864         ['input_1[0][0]']                
                                                                                                  
 Conv2d_1a_3x3_BatchNorm (Batch  (None, 79, 79, 32)  96          ['Conv2d_1a_3x3[0][0]']          
 Normalization)                                                                                   
                                                                                

The Triplet Loss

In [39]:
def triplet_loss(y_true, y_pred, alpha=0.2):
    """
    Implementation of the triplet loss
    
    :param y_true -- true labels, required when you define a loss in Keras, you don't need it in this function.
    :param y_pred -- python list containing three objects:
            anchor -- the encodings for the anchor images, of shape (None, 128)
            positive -- the encodings for the positive images, of shape (None, 128)
            negative -- the encodings for the negative images, of shape (None, 128)
    
    returns:
    loss -- real number, value of the loss
    """
    # unpacking the output of the as anchor, positive and negative
    anchor, positive, negative = y_pred[0], y_pred[1], y_pred[2]
    # postive distance
    pos_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, positive)), axis=-1)
    # negative distance
    neg_dist = tf.reduce_sum(tf.square(tf.subtract(anchor, negative)), axis=-1)
    # subtract previous distances and add alpha
    basic_loss = tf.maximum(tf.add(tf.subtract(pos_dist,neg_dist),alpha),0)
    # final loss
    loss = tf.reduce_sum(basic_loss)

    return loss


In [40]:
tf.random.set_seed(1)
y_true = (None, None, None) # It is not used
y_pred = (tf.keras.backend.random_normal([3, 128], mean=6, stddev=0.1, seed = 1),
          tf.keras.backend.random_normal([3, 128], mean=1, stddev=1, seed = 1),
          tf.keras.backend.random_normal([3, 128], mean=3, stddev=4, seed = 1))
loss = triplet_loss(y_true, y_pred)

print("loss = " + str(loss))



loss = tf.Tensor(527.2598, shape=(), dtype=float32)


# Face Verification

In [41]:
# face recognition
FRmodel = model

In [42]:
def img_to_encoding(image_path, model):
  """
  maps an image to its corresponding encoding by doing a forward pass on the model

  :param image_path: string, path of the image to be encoded
  :param model: tf.keras.model, model to be used for encoding of the image

  returns:
  embedding: ndarray of shape (1, 128). Encoding of the image.
  """
  # load the image
  img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
  # Normalization
  img = np.around(np.array(img) / 255.0, decimals=12)
  # the input to the model is in the form of (None, 160, 160, 3). 
  # Thus, expanding the dimension
  x_train = np.expand_dims(img, axis=0)
  # Forward pass
  embedding = model.predict_on_batch(x_train)
  return embedding / np.linalg.norm(embedding, ord=2)

In [44]:
# creating a dictionary of the images mapping each person's image to a 128-dimensional encoding vector
database = {}
# image directory
image_dir = os.path.join(ROOT_DIR, 'images')
database["danielle"] = img_to_encoding(os.path.join(image_dir, "danielle.png"), FRmodel)
database["younes"] = img_to_encoding(os.path.join(image_dir, "younes.jpg"), FRmodel)
database["tian"] = img_to_encoding(os.path.join(image_dir, "tian.jpg"), FRmodel)
database["andrew"] = img_to_encoding(os.path.join(image_dir, "andrew.jpg"), FRmodel)
database["kian"] = img_to_encoding(os.path.join(image_dir, "kian.jpg"), FRmodel)
database["dan"] = img_to_encoding(os.path.join(image_dir, "dan.jpg"), FRmodel)
database["sebastiano"] = img_to_encoding(os.path.join(image_dir, "sebastiano.jpg"), FRmodel)
database["bertrand"] = img_to_encoding(os.path.join(image_dir, "bertrand.jpg"), FRmodel)
database["kevin"] = img_to_encoding(os.path.join(image_dir, "kevin.jpg"), FRmodel)
database["felix"] = img_to_encoding(os.path.join(image_dir, "felix.jpg"), FRmodel)
database["benoit"] = img_to_encoding(os.path.join(image_dir, "benoit.jpg"), FRmodel)
database["arnaud"] = img_to_encoding(os.path.join(image_dir, "arnaud.jpg"), FRmodel)

In [52]:
# verify the identity
def verify(image_path, identity, database, model):
  """
  Function that verifies if the person on the "image path" is the "identity"

  :param image_path: path to an image
  :param identity: string, name of the person you'd like to verify the identity. Has to be an employee who works in the office.
  :param database: python dictionary mapping names of allowed people's names (strings) to their encodings (vectors).
  :param model: Inception model instance in Keras

  returns:
  dist -- distance between the image_path and the image of "identity" in the database.
  door_open -- True, if the door should open. False otherwise.
  """
  #Step 1: Compute the encoding for the image. 
  encoding = img_to_encoding(image_path, model)
  # Step 2: Compute distance with identity's image (≈ 1 line)
  dist = np.linalg.norm(encoding - database[identity])
  # Step 3: Open the door if dist < 0.7, else don't open (≈ 3 lines)
  if dist < 0.7:
      print("It's " + str(identity) + ", welcome in!")
      door_open = True
  else:
      print("It's not " + str(identity) + ", please go away")
      door_open = False
      
  return dist, door_open

In [ ]:


# GRADED FUNCTION: who_is_it

def who_is_it(image_path, database, model):
  """
  Implements face recognition for the office by finding who is the person on the image_path image.

  :param image_path: path to an image
  :param database: database containing image encodings along with the name of the person on the image
  :param model: Inception model instance in Keras

  returns:
  min_dist: the minimum distance between image_path encoding and the encodings from the database
  identity: string, the name prediction for the person on image_path
  """
  # find the encoding of the image
  encoding = img_to_encoding(image_path, model)

  # find the closest encoding
  min_dist = 100
  
  #Loop over the database dictionary's names and encodings.
  for (name, db_enc) in database.items():
      
      # Compute L2 distance between the target "encoding" and the current db_enc from the database. (≈ 1 line)
      dist = np.linalg.norm(encoding - db_enc)

      # If this distance is less than the min_dist, then set min_dist to dist, and identity to name. (≈ 3 lines)
      if dist < min_dist:
          min_dist = dist
          identity = name
  
  # YOUR CODE ENDS HERE
  
  if min_dist > 0.7:
      print("Not in the database.")
  else:
      print ("it's " + str(identity) + ", the distance is " + str(min_dist))
      
  return min_dist, identity

